In [ ]:
!date "+[%F %R:%S] [INIT] HelloNSMC (using $CONDA_PREFIX)"
import time
t0 = time.time()

In [ ]:
!pip install pytorch_transformers

In [ ]:
import os
if not os.path.exists("../dat/nsmc/"):
    !git clone https://github.com/e9t/nsmc.git "../dat/nsmc"
else:
    print("Dataset already downloaded: nsmc")

In [ ]:
!date "+[%F %R:%S] [DONE] Environment & Dataset Installation"
print(f"td={time.time() - t0:.3f}")
t0 = time.time()

In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from pytorch_transformers import BertTokenizer, BertForSequenceClassification
from torch.optim import Adam
import torch.nn.functional as F

In [ ]:
train_df = pd.read_csv('../dat/nsmc/ratings_train.txt', sep='\t')
test_df = pd.read_csv('../dat/nsmc/ratings_test.txt', sep='\t')
train_df.head(3)

In [ ]:
train_df.dropna(inplace=True)
test_df.dropna(inplace=True)

train_df = train_df.sample(frac=0.4, random_state=999)
test_df = test_df.sample(frac=0.4, random_state=999)

In [ ]:
class NsmcDataset(Dataset):
    """ Naver Sentiment Movie Corpus Dataset """
    def __init__(self, df):
        self.df = df

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        text = self.df.iloc[idx, 1]
        label = self.df.iloc[idx, 2]
        return text, label

In [ ]:
nsmc_train_dataset = NsmcDataset(train_df)
train_loader = DataLoader(nsmc_train_dataset, batch_size=2, shuffle=True, num_workers=2)

In [ ]:
device = torch.device("cuda")
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased')
model.to(device)

In [ ]:
!date "+[%F %R:%S] [DONE] Pretrained Model Loading"
print(f"td={time.time() - t0:.3f}")
t0 = time.time()

In [ ]:
optimizer = Adam(model.parameters(), lr=1e-6)

epochs = 2
itr = 1
p_itr = 500
total_loss = 0
total_len = 0
total_correct = 0

model.train()
for epoch in range(epochs):

    for text, label in train_loader:
        optimizer.zero_grad()

        encoded_list = [tokenizer.encode(t, add_special_tokens=True) for t in text]
        padded_list =  [e + [0] * (512-len(e)) for e in encoded_list]
        sample = torch.tensor(padded_list)
        sample, label = sample.to(device), label.to(device)
        labels = torch.tensor(label)
        outputs = model(sample, labels=labels)
        loss, logits = outputs

        pred = torch.argmax(F.softmax(logits), dim=1)
        correct = pred.eq(labels)
        total_correct += correct.sum().item()
        total_len += len(labels)
        total_loss += loss.item()
        loss.backward()
        optimizer.step()

        if itr % p_itr == 0:
            !date "+[%F %R:%S] Training..."
            print(f"[Epoch {epoch+1}/{epochs}] Iteration {itr} -> Train Loss: {total_loss/p_itr:.4f}, Accuracy: {total_correct/total_len:.4f}")
            total_loss = 0
            total_len = 0
            total_correct = 0

        itr+=1

In [ ]:
!date "+[%F %R:%S] [DONE] Model Fine-tuning"
print(f"td={time.time() - t0:.3f}")
t0 = time.time()

In [ ]:
# evaluation
model.eval()

nsmc_eval_dataset = NsmcDataset(test_df)
eval_loader = DataLoader(nsmc_eval_dataset, batch_size=2, shuffle=False, num_workers=2)

total_loss = 0
total_len = 0
total_correct = 0

for text, label in eval_loader:
    encoded_list = [tokenizer.encode(t, add_special_tokens=True) for t in text]
    padded_list =  [e + [0] * (512-len(e)) for e in encoded_list]
    sample = torch.tensor(padded_list)
    sample, label = sample.to(device), label.to(device)
    labels = torch.tensor(label)
    outputs = model(sample, labels=labels)
    _, logits = outputs

    pred = torch.argmax(F.softmax(logits), dim=1)
    correct = pred.eq(labels)
    total_correct += correct.sum().item()
    total_len += len(labels)

print(f"Test accuracy: {total_correct / total_len:.4f}")

In [ ]:
!date "+[%F %R:%S] [EXIT] HelloNSMC (using $CONDA_PREFIX)"
print(f"td={time.time() - t0:.3f}")